In [1]:
import flair.datasets
from pathlib import Path
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import TokenEmbeddings, BertEmbeddings,  ELMoEmbeddings, FlairEmbeddings, WordEmbeddings, PooledFlairEmbeddings, StackedEmbeddings, CharacterEmbeddings, CharLMEmbeddings
from flair.models import LanguageModel
from gensim.models.keyedvectors import KeyedVectors
from typing import List
import os

In [2]:
# Disease
# NCBI_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/NCBI-disease-IOB'
#BioNLP13PC_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BioNLP13PC-IOB'
# BC5CDR_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC5CDR-disease-IOB'
# Genes
BC2GM_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC2GM-IOB'
# JNLPBA_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/JNLPBA'



# define columns
columns = {0: 'text', 1: 'ner'}

In [3]:
# NCBI = ColumnCorpus(NCBI_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)

# BioNLP13PC = ColumnCorpus(BioNLP13PC_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)

# BC5CDR = ColumnCorpus(BC5CDR_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)


BC2GM = ColumnCorpus(BC2GM_data_folder, columns, 
                              train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)


# JNLPBA = ColumnCorpus(JNLPBA_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)

2019-09-18 17:20:05,579 Reading data from /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC2GM-IOB
2019-09-18 17:20:05,583 Train: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC2GM-IOB/train.tsv
2019-09-18 17:20:05,584 Dev: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC2GM-IOB/devel.tsv
2019-09-18 17:20:05,585 Test: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC2GM-IOB/test.tsv


In [4]:
from flair.data import MultiCorpus
# multi_corpus = MultiCorpus([NCBI, BC5CDR])
multi_corpus = MultiCorpus([BC2GM])

In [5]:
#corpus: TaggedCorpus = read_data()
print(multi_corpus)

Corpus: 12574 train + 2519 dev + 5038 test sentences


In [6]:
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = multi_corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

[b'<unk>', b'O', b'B-GENE', b'I-GENE', b'<START>', b'<STOP>']


In [7]:
# embedding_types: List[TokenEmbeddings] = [

#     WordEmbeddings('glove'),

# #     pretrained_word2vec,

#     # comment in this line to use character embeddings
#     # CharacterEmbeddings(),

#     # comment in these lines to use contextual string embeddings
#     FlairEmbeddings('news-forward'),
#     FlairEmbeddings('news-backward'),
# ]

# word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('pubmed-forward'), 
# FlairEmbeddings('pubmed-backward'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

# elmo_embeddings_pubmed = ELMoEmbeddings('pubmed')

embedding_types: List[TokenEmbeddings] = [
                   WordEmbeddings('glove'),
                   # comment in flair embeddings for state-of-the-art results
                   FlairEmbeddings('news-forward'),
                   FlairEmbeddings('news-backward'),
                   CharacterEmbeddings(), 
                   PooledFlairEmbeddings('pubmed-backward', pooling='max'),
                   PooledFlairEmbeddings('pubmed-forward', pooling='max'),
#                    BertEmbeddings('bert-large-cased'),
#                    elmo_embeddings_pubmed
                   ]

In [8]:
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

In [9]:
# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

In [10]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, multi_corpus)

In [ ]:
# 7. start training
# data_path = '/nfs/gns/literature/Santosh_Tirunagari/'
# if os.path.isdir(data_path+'/Models/') == False:
#     os.mkdir(data_path+'/Models/ner_flair_model')

In [11]:
trainer.train('../flair_models/ner/multi_bio_ner_model_gene/v01',
              learning_rate=0.1,
              mini_batch_size=128,
              max_epochs=150)

2019-09-18 17:25:10,010 ----------------------------------------------------------------------------------------------------
2019-09-18 17:25:10,011 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_3): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
    (list_embedding_4): PooledFlairEmbeddings(
      (context_embeddings): FlairEmbed

2019-09-18 18:02:18,802 epoch 4 - iter 63/99 - loss 2.04061468 - samples/sec: 22.65
2019-09-18 18:03:06,093 epoch 4 - iter 72/99 - loss 2.01696462 - samples/sec: 25.51
2019-09-18 18:03:57,385 epoch 4 - iter 81/99 - loss 2.04778510 - samples/sec: 23.87
2019-09-18 18:04:48,138 epoch 4 - iter 90/99 - loss 2.04033269 - samples/sec: 23.69
2019-09-18 18:05:26,289 ----------------------------------------------------------------------------------------------------
2019-09-18 18:05:26,292 EPOCH 4 done: loss 2.0246 - lr 0.1000
2019-09-18 18:06:51,473 DEV : loss 1.4483518600463867 - score 0.73
2019-09-18 18:06:56,180 BAD EPOCHS (no improvement): 0
2019-09-18 18:07:05,492 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-18 18:07:13,506 epoch 5 - iter 0/99 - loss 1.50153124 - samples/sec: 198.95
2019-09-18 18:08:00,420 epoch 5 - iter 9/99 - loss 1.75732604 - samples/sec: 25.82

2019-09-18 19:01:46,665 epoch 10 - iter 27/99 - loss 1.47718882 - samples/sec: 24.10
2019-09-18 19:02:32,508 epoch 10 - iter 36/99 - loss 1.49297701 - samples/sec: 26.14
2019-09-18 19:03:22,587 epoch 10 - iter 45/99 - loss 1.45798732 - samples/sec: 23.86
2019-09-18 19:04:10,736 epoch 10 - iter 54/99 - loss 1.43759565 - samples/sec: 24.88
2019-09-18 19:05:02,606 epoch 10 - iter 63/99 - loss 1.45691635 - samples/sec: 23.04
2019-09-18 19:05:50,527 epoch 10 - iter 72/99 - loss 1.46017006 - samples/sec: 25.11
2019-09-18 19:06:36,695 epoch 10 - iter 81/99 - loss 1.45726198 - samples/sec: 25.98
2019-09-18 19:07:26,763 epoch 10 - iter 90/99 - loss 1.44912435 - samples/sec: 23.61
2019-09-18 19:08:06,087 ----------------------------------------------------------------------------------------------------
2019-09-18 19:08:06,089 EPOCH 10 done: loss 1.4798 - lr 0.1000
2019-09-18 19:09:31,598 DEV : loss 1.2221107482910156 - score 0.7691
2019-09-18 19:09:36,884 BAD EPOCHS (no improvement): 0
2019-09-

train mode resetting embeddings
train mode resetting embeddings
2019-09-18 20:02:09,525 epoch 16 - iter 0/99 - loss 0.97195143 - samples/sec: 225.87
2019-09-18 20:02:57,705 epoch 16 - iter 9/99 - loss 1.22129104 - samples/sec: 25.00
2019-09-18 20:03:43,169 epoch 16 - iter 18/99 - loss 1.21084200 - samples/sec: 26.21
2019-09-18 20:04:29,776 epoch 16 - iter 27/99 - loss 1.18018074 - samples/sec: 25.79
2019-09-18 20:05:15,687 epoch 16 - iter 36/99 - loss 1.16750878 - samples/sec: 26.24
2019-09-18 20:06:06,081 epoch 16 - iter 45/99 - loss 1.17601496 - samples/sec: 23.70
2019-09-18 20:06:52,546 epoch 16 - iter 54/99 - loss 1.16715619 - samples/sec: 25.65
2019-09-18 20:07:40,912 epoch 16 - iter 63/99 - loss 1.15459149 - samples/sec: 24.93
2019-09-18 20:08:29,259 epoch 16 - iter 72/99 - loss 1.14916173 - samples/sec: 24.69
2019-09-18 20:09:19,014 epoch 16 - iter 81/99 - loss 1.15804624 - samples/sec: 24.22
2019-09-18 20:10:07,607 epoch 16 - iter 90/99 - loss 1.16856285 - samples/sec: 24.71
20

2019-09-18 21:02:36,537 ----------------------------------------------------------------------------------------------------
2019-09-18 21:02:36,539 EPOCH 21 done: loss 1.0180 - lr 0.0500
2019-09-18 21:04:02,227 DEV : loss 1.001797080039978 - score 0.808
2019-09-18 21:04:06,679 BAD EPOCHS (no improvement): 0
2019-09-18 21:04:15,971 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-18 21:04:24,386 epoch 22 - iter 0/99 - loss 1.04306746 - samples/sec: 197.65
2019-09-18 21:05:14,666 epoch 22 - iter 9/99 - loss 1.12951232 - samples/sec: 24.18
2019-09-18 21:06:01,916 epoch 22 - iter 18/99 - loss 1.07259786 - samples/sec: 25.03
2019-09-18 21:06:51,609 epoch 22 - iter 27/99 - loss 1.05581762 - samples/sec: 24.22
2019-09-18 21:07:40,897 epoch 22 - iter 36/99 - loss 1.01030336 - samples/sec: 24.17
2019-09-18 21:08:25,915 epoch 22 - iter 45/99 - loss 0.98755570 - samples/sec

...

2019-09-19 02:59:26,604 epoch 56 - iter 0/99 - loss 0.92873305 - samples/sec: 219.27
2019-09-19 03:00:15,983 epoch 56 - iter 9/99 - loss 0.83472705 - samples/sec: 24.11
2019-09-19 03:01:05,244 epoch 56 - iter 18/99 - loss 0.80704753 - samples/sec: 24.28
2019-09-19 03:01:50,947 epoch 56 - iter 27/99 - loss 0.80219942 - samples/sec: 26.48
2019-09-19 03:02:38,503 epoch 56 - iter 36/99 - loss 0.78332454 - samples/sec: 24.91
2019-09-19 03:03:27,398 epoch 56 - iter 45/99 - loss 0.76110818 - samples/sec: 24.18
2019-09-19 03:04:16,874 epoch 56 - iter 54/99 - loss 0.75543125 - samples/sec: 24.46
2019-09-19 03:05:04,128 epoch 56 - iter 63/99 - loss 0.73822810 - samples/sec: 25.58
2019-09-19 03:05:54,693 epoch 56 - iter 72/99 - loss 0.74303218 - samples/sec: 24.14
2019-09-19 03:06:44,209 epoch 56 - iter 81/99 - loss 0.74965994 - samples/sec: 23.89
2019-09-19 03:07:32,544 epoch 56 - iter 90/99 - loss 0.74272936 - samples/sec: 25.26
2019-09-19 03:08:11,642 ------------------------------------------

2019-09-19 04:00:21,618 EPOCH 61 done: loss 0.7343 - lr 0.0016
2019-09-19 04:01:48,277 DEV : loss 0.9082862734794617 - score 0.8233
Epoch    60: reducing learning rate of group 0 to 7.8125e-04.
2019-09-19 04:01:53,409 BAD EPOCHS (no improvement): 4
2019-09-19 04:01:53,416 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-19 04:02:01,808 epoch 62 - iter 0/99 - loss 0.69590878 - samples/sec: 215.40
2019-09-19 04:02:48,496 epoch 62 - iter 9/99 - loss 0.76747209 - samples/sec: 25.46
2019-09-19 04:03:42,678 epoch 62 - iter 18/99 - loss 0.77910552 - samples/sec: 21.88
2019-09-19 04:04:32,304 epoch 62 - iter 27/99 - loss 0.75891512 - samples/sec: 24.42
2019-09-19 04:05:20,870 epoch 62 - iter 36/99 - loss 0.75124520 - samples/sec: 24.78
2019-09-19 04:06:11,764 epoch 62 - iter 45/99 - loss 0.74726205 - samples/sec: 23.72
2019-09-19 04:07:00,295 epoch 62 - iter 54/99 - loss 

2019-09-19 04:59:49,665 epoch 67 - iter 63/99 - loss 0.75119135 - samples/sec: 24.97
2019-09-19 05:00:39,168 epoch 67 - iter 72/99 - loss 0.74486063 - samples/sec: 24.01
2019-09-19 05:01:27,057 epoch 67 - iter 81/99 - loss 0.74517929 - samples/sec: 25.19
2019-09-19 05:02:13,759 epoch 67 - iter 90/99 - loss 0.73432544 - samples/sec: 25.62
2019-09-19 05:02:53,443 ----------------------------------------------------------------------------------------------------
2019-09-19 05:02:53,445 EPOCH 67 done: loss 0.7312 - lr 0.0004
2019-09-19 05:04:20,090 DEV : loss 0.9083898663520813 - score 0.8228
2019-09-19 05:04:24,354 BAD EPOCHS (no improvement): 2
2019-09-19 05:04:24,361 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-19 05:04:32,108 epoch 68 - iter 0/99 - loss 1.08114791 - samples/sec: 240.97
2019-09-19 05:05:18,064 epoch 68 - iter 9/99 - loss 0.77104665 - samples/s

2019-09-19 05:58:34,194 epoch 73 - iter 18/99 - loss 0.69826989 - samples/sec: 25.88
2019-09-19 05:59:24,311 epoch 73 - iter 27/99 - loss 0.73644595 - samples/sec: 23.97
2019-09-19 06:00:13,314 epoch 73 - iter 36/99 - loss 0.72837725 - samples/sec: 24.62
2019-09-19 06:00:59,330 epoch 73 - iter 45/99 - loss 0.72540881 - samples/sec: 26.24
2019-09-19 06:01:48,679 epoch 73 - iter 54/99 - loss 0.72834587 - samples/sec: 24.27
2019-09-19 06:02:38,986 epoch 73 - iter 63/99 - loss 0.73729625 - samples/sec: 23.69
2019-09-19 06:03:23,631 epoch 73 - iter 72/99 - loss 0.72643295 - samples/sec: 26.81
2019-09-19 06:04:15,732 epoch 73 - iter 81/99 - loss 0.73008469 - samples/sec: 22.79
2019-09-19 06:05:03,061 epoch 73 - iter 90/99 - loss 0.73247236 - samples/sec: 25.50
2019-09-19 06:05:42,993 ----------------------------------------------------------------------------------------------------
2019-09-19 06:05:42,996 EPOCH 73 done: loss 0.7305 - lr 0.0002
2019-09-19 06:07:08,435 DEV : loss 0.9082191586

{'test_score': 0.8086,
 'dev_score_history': [0.4185,
  0.6684,
  0.7081,
  0.73,
  0.7126,
  0.7424,
  0.7427,
  0.7297,
  0.7474,
  0.7691,
  0.7695,
  0.7726,
  0.7798,
  0.7783,
  0.7962,
  0.7896,
  0.7935,
  0.7931,
  0.7948,
  0.8049,
  0.808,
  0.8102,
  0.8054,
  0.8105,
  0.8088,
  0.8063,
  0.8172,
  0.8115,
  0.8123,
  0.811,
  0.8123,
  0.8157,
  0.8231,
  0.8203,
  0.8181,
  0.8145,
  0.8209,
  0.8224,
  0.8225,
  0.8234,
  0.8207,
  0.8251,
  0.8207,
  0.824,
  0.8266,
  0.8218,
  0.8203,
  0.8249,
  0.8257,
  0.8229,
  0.8242,
  0.823,
  0.8234,
  0.8247,
  0.8214,
  0.8229,
  0.8234,
  0.8235,
  0.8232,
  0.8235,
  0.8233,
  0.8232,
  0.8235,
  0.8237,
  0.823,
  0.8225,
  0.8228,
  0.8228,
  0.823,
  0.8225,
  0.8227,
  0.8223,
  0.8228],
 'train_loss_history': [5.622059407860342,
  2.792812298042606,
  2.3380070551477297,
  2.024633912125019,
  1.8088088685815984,
  1.713933330593687,
  1.6616654444222498,
  1.4986572795444064,
  1.4854647426894216,
  1.4798410661292

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence, Token
from flair.models import SequenceTagger
# load the model you trained
# gene_model = SequenceTagger.load('../flair_models/ner/multi_bio_ner_model_gene/best-model.pt')
# disease_model = SequenceTagger.load('../flair_models/ner/multi_bio_ner_model_disease/best-model.pt')


# create example sentence
sentence= 'Conditions for production, and some characteristics, of mycobacterial growth inhibitory factor produced by spleen cells from mice immunized with viable cells of the attenuated H37Ra strain of Mycobacterium tuberculosis.'

gene_sentence = Sentence(sentence)
gene_model.predict(gene_sentence)

disease_sentence = Sentence(sentence)
disease_model.predict(disease_sentence)

In [ ]:
str(gene_sentence.get_spans('ner')).split(':')[1].replace('"','').replace(']','').replace('>','').strip()

In [ ]:
str(disease_sentence.get_spans('ner')).split(':')[1].replace('"','').replace(']','').replace('>','').strip()

In [ ]:
# from flair.models import TextClassifier
# from flair.data import Sentence, Token

# # load the model you trained
# # disease_model = SequenceTagger.load('../flair_models/ner/multi_bio_ner_model_disease/best-model.pt')

# # create example sentence
# xx= 'For example, the chemical inhibition of IL4I1 activity may represent a new adjuvant strategy for the treatment of cancer by restoring specific anti-tumor immune responses.'

# xx1 = 'These results suggested that during ischemic cardiomyopathy, initially neutrophil proteinase activates latent myocardial MMP which can degrade ECM, which continuously degrades if not controlled by TIMP, leading to ventricular dilatation and dysfunction.'

# xx2= 'GR3027 antagonizes GABAA receptor potentiating neurosteroids and restores spatial learning and motor coordination in rats with chronic hyperammonemia and hepatic encephalopathy'

# sentence = Sentence(xx2)

# # predict tags and print
# disease_model.predict(sentence)

# print(sentence.to_tagged_string())